In [43]:
# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = pyplot.imread(filename)
#     # create the detector, using default weights
#     detector = MTCNN()
#     # detect faces in the image
#     results = detector.detect_faces(pixels)
#     # extract the bounding box from the first face
#     x1, y1, width, height = results[0]['box']
#     x2, y2 = x1 + width, y1 + height
#     # extract the face
#     face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(pixels)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

model = None
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    global model
    if not model:
        model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    print(known_embedding,'\n', candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
        return True
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
        return False

In [1]:
import pathlib

SAMPLE_SIZE = 10
BASE_PATH = pathlib.Path('../data/AFDB')

unmasked_dataset_path = BASE_PATH / 'AFDB_face_dataset' / 'AFDB_face_dataset'
masked_dataset_path = BASE_PATH / 'AFDB_masked_face_dataset' / 'AFDB_masked_face_dataset'

In [48]:
unmasked_caizhuoyan = sorted(list((unmasked_dataset_path / 'caizhuoyan').iterdir()))[0]
masked_caizhuoyan = sorted(list((masked_dataset_path / 'caizhuoyan').iterdir()))[0]
unmasked_image_path = str(unmasked_aidai_image)
masked_image_path = str(masked_aidai)
    
unmasked_features, masked_features = get_embeddings([unmasked_image_path, masked_image_path])
caizhuoyan_matched = is_match(unmasked_features, masked_features)

[0.19216926 0.23195851 0.04736145 ... 2.9882371  0.1381264  7.7278214 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.626 > 0.500)


In [38]:
results = []

unmasked_people = sorted(list(unmasked_dataset_path.iterdir()))
for unmasked_person in unmasked_people:
    masked_person = masked_dataset_path / unmasked_person.name
    if not masked_person.is_dir():
        continue
    unmasked_image_path = str(sorted(list(unmasked_person.iterdir()))[0])
    masked_image_path = str(sorted(list(masked_person.iterdir()))[0])
    
    try:
        unmasked_features, masked_features = get_embeddings([unmasked_image_path, masked_image_path])
        matched = is_match(unmasked_features, masked_features)
    except Exception as e:
        print(e)
        continue
    
    results.append([unmasked_person.name, matched])

[6.4286904  0.891206   0.         ... 0.01346317 3.2546344  0.03368115] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.634 > 0.500)
[1.1805403  0.92346495 0.03786647 ... 0.15240227 7.3179674  0.        ] 
 [0.         0.         0.08890074 ... 0.         0.         0.01308496]
>face is NOT a Match (0.770 > 0.500)
[0.3517487  3.298802   0.2040287  ... 9.026708   0.         0.19876951] 
 [0.03066392 0.         0.6384587  ... 0.         0.         0.11664617]
>face is NOT a Match (0.646 > 0.500)
[0.         0.0255706  7.181293   ... 0.         0.01174408 0.        ] 
 [0.0596545  0.00369168 0.01057714 ... 0.         0.         0.        ]
>face is NOT a Match (0.588 > 0.500)
[0.63419026 2.816073   0.614268   ... 1.0322208  2.147673   2.193346  ] 
 [0.0366322  0.04634538 0.         ... 0.         0.         0.        ]
>face is NOT a Match (0.605 > 0.500)
[0.         1.1360775  2.2802646  ... 3.1888032  0.80064535 0.93340707] 
 [0.0000000e+00 1.

[3.4405403  0.86093396 1.3449848  ... 0.1106395  0.9745882  0.26446444] 
 [0.01480744 0.29259345 0.         ... 0.00332432 0.2212872  3.2772024 ]
>face is NOT a Match (0.559 > 0.500)
[ 4.340182    0.1441852   0.         ...  2.3855038   0.10941296
 13.149665  ] 
 [2.8658142 0.5542135 0.        ... 0.        0.        3.3316092]
>face is NOT a Match (0.504 > 0.500)
[1.3150723  0.         4.8974986  ... 0.19827919 1.5633421  0.        ] 
 [0.         0.         0.         ... 0.         0.04260783 0.        ]
>face is NOT a Match (0.627 > 0.500)
[0.        0.        5.958615  ... 3.8417501 0.        0.       ] 
 [6.940106   0.50007373 0.         ... 0.         3.278783   1.4214029 ]
>face is NOT a Match (0.695 > 0.500)
Cannot handle this data type: (1, 1, 3), <f4
Cannot handle this data type: (1, 1, 3), <f4
[0.         0.5408349  1.0688176  ... 0.04779039 4.69431    0.        ] 
 [0.3261583  1.4339375  0.98360384 ... 0.         2.2748446  1.6425405 ]
>face is NOT a Match (0.590 > 0.500)


[1.2211624  6.830986   0.31558865 ... 1.2932364  5.918587   1.0611547 ] 
 [7.5236263  8.229839   0.         ... 0.01521511 0.9500682  0.7145843 ]
>face is a Match (0.453 <= 0.500)
[2.0914965  1.3172319  0.         ... 3.3567567  1.564881   0.26475197] 
 [3.181191   1.1643738  0.07545354 ... 0.         0.09542016 0.        ]
>face is NOT a Match (0.503 > 0.500)
[6.7830653  0.20215577 0.         ... 0.         0.57619244 1.8354853 ] 
 [1.3542125  0.         0.         ... 0.         0.00497279 0.0190681 ]
>face is NOT a Match (0.659 > 0.500)
[0.         0.29920015 0.40447694 ... 0.         0.         0.        ] 
 [0.48754746 0.15140268 0.         ... 0.         0.         0.        ]
>face is NOT a Match (0.731 > 0.500)
[0.         0.13491407 0.         ... 0.19428748 2.16878    0.6082417 ] 
 [0.00494263 0.00933099 0.         ... 0.         0.32318088 1.5920613 ]
>face is a Match (0.481 <= 0.500)
[0.        4.1480503 0.        ... 0.        1.2560931 0.5044728] 
 [4.2276816  0.12751527 

[0.1834668  0.14503808 0.         ... 3.4827595  5.833995   0.01428341] 
 [0.14626503 0.04452566 0.         ... 0.         0.         0.        ]
>face is NOT a Match (0.545 > 0.500)
[2.852189   0.21683538 0.33294395 ... 0.         0.         1.813054  ] 
 [0.36390582 0.         0.         ... 0.         0.04742233 0.        ]
>face is NOT a Match (0.680 > 0.500)
[0.5988705 0.6963931 5.2400966 ... 3.3817668 0.        1.3193034] 
 [2.39454   0.1714042 0.        ... 0.        0.        1.8686259]
>face is NOT a Match (0.544 > 0.500)
[0.         0.         0.03147268 ... 0.7473269  0.0456104  0.13188517] 
 [0.02901041 0.         2.7612722  ... 0.         1.6595305  0.        ]
>face is a Match (0.494 <= 0.500)
[0.        0.7000214 1.4862161 ... 0.        0.        0.       ] 
 [0.         0.         0.05302775 ... 0.10842998 0.         0.        ]
>face is NOT a Match (0.530 > 0.500)
[7.1286716  1.3429147  0.         ... 0.24059156 3.0328696  0.11476415] 
 [0.        0.        0.        .

[0.         1.6349018  0.01263563 ... 1.5226601  0.15580703 5.2795424 ] 
 [1.9504212  0.7644711  0.30789793 ... 0.         0.         0.46330017]
>face is NOT a Match (0.659 > 0.500)
[0.93741715 0.         3.7025993  ... 4.0033646  0.10501918 0.        ] 
 [0.2272414  0.         0.03075172 ... 0.         1.412968   0.        ]
>face is NOT a Match (0.572 > 0.500)
[0.         3.4060967  0.2598771  ... 4.704505   0.65883046 0.5307957 ] 
 [0.59787923 0.         0.         ... 0.         0.56978536 0.        ]
>face is NOT a Match (0.596 > 0.500)
[0.06697813 0.03488377 9.623208   ... 1.154281   0.         0.        ] 
 [3.2248595e-03 9.0668395e-02 2.7442534e+00 ... 0.0000000e+00 0.0000000e+00
 6.8264885e+00]
>face is a Match (0.457 <= 0.500)
[0.        2.666992  2.994815  ... 3.7462218 0.7349989 2.2276974] 
 [0.         0.44662452 0.         ... 0.         0.         0.        ]
>face is NOT a Match (0.708 > 0.500)
[5.5440025  0.         0.00853347 ... 0.         1.9157383  4.114839  ] 
 [

[0.1726045  0.         7.8071737  ... 0.2322376  0.62233734 0.05754549] 
 [2.0603902  4.574886   0.         ... 0.22737157 0.01744909 3.3272371 ]
>face is NOT a Match (0.766 > 0.500)
[0.7291793  0.29643503 0.         ... 1.6424626  4.51089    1.7646835 ] 
 [2.377045   0.37242493 0.         ... 0.01729439 2.1478093  0.        ]
>face is a Match (0.463 <= 0.500)
[0.09329631 0.34406927 3.6832955  ... 2.9037778  0.21904139 1.8576697 ] 
 [2.3327143  0.5563291  0.         ... 0.         0.         0.27340886]
>face is NOT a Match (0.742 > 0.500)
[0.         0.15415123 1.1018368  ... 0.5592272  0.02931443 1.359632  ] 
 [0.93258065 0.23364145 0.         ... 0.         5.1547527  0.5501941 ]
>face is NOT a Match (0.510 > 0.500)
[0.00820076 0.         0.16273525 ... 0.         1.0998496  4.0305424 ] 
 [0.         0.3774335  0.         ... 0.         0.         0.28742015]
>face is NOT a Match (0.762 > 0.500)
[1.2762535e-03 1.6609648e+00 7.3005551e-01 ... 4.1687884e+00 2.3310682e-01
 2.4858291e-0

[0.14614828 0.27996758 0.         ... 1.2847757  6.1298194  0.38565892] 
 [0.10696536 0.         0.         ... 0.         0.         0.        ]
>face is NOT a Match (0.739 > 0.500)
[5.264045   0.         0.9727776  ... 0.         0.01993653 1.3881426 ] 
 [3.1199603  0.         1.9596064  ... 0.         0.00908726 0.        ]
>face is a Match (0.315 <= 0.500)
[2.4472973  0.0209725  0.         ... 0.0970367  0.75817513 2.4207964 ] 
 [12.768847    0.01301058  0.         ...  0.01458947  0.
  0.        ]
>face is a Match (0.445 <= 0.500)
[0.3161718  0.24245009 2.9121137  ... 0.         1.1529458  0.07671349] 
 [0.13032864 0.         0.         ... 0.         0.         0.37427026]
>face is NOT a Match (0.628 > 0.500)
[0.        0.6438652 0.3510942 ... 2.254903  4.674582  0.       ] 
 [0.         0.         0.         ... 0.00310182 0.         0.8870665 ]
>face is NOT a Match (0.686 > 0.500)
[0.00597141 0.         0.04591897 ... 0.7504981  2.352805   3.619437  ] 
 [0.         1.9066545  0

[1.4722784  0.         0.11923115 ... 0.26962087 0.2123719  0.01481396] 
 [6.6592655  0.         0.         ... 0.10642643 0.39781594 0.        ]
>face is NOT a Match (0.521 > 0.500)
[2.829068   1.2357752  0.25236055 ... 0.         0.5528004  6.538865  ] 
 [0.37740824 0.0210804  0.         ... 0.00709254 0.95751774 0.        ]
>face is NOT a Match (0.755 > 0.500)
[1.2620622e+00 7.6624326e-04 0.0000000e+00 ... 1.1323003e-01 3.1368802e+00
 7.2209859e-01] 
 [0.97046846 0.16166672 0.         ... 0.01164011 0.12199143 0.        ]
>face is NOT a Match (0.726 > 0.500)
[0.0000000e+00 1.5880916e-03 2.2257750e+00 ... 3.5313509e+00 4.2165172e-01
 1.6705754e-01] 
 [2.231078   0.23417671 2.4227755  ... 0.         2.1478248  0.0552107 ]
>face is NOT a Match (0.631 > 0.500)
[0.         2.2891278  0.         ... 0.13588597 0.         0.15807812] 
 [0.0000000e+00 1.0998505e+00 1.8124210e-02 ... 2.0430915e-03 3.3074915e-01
 3.0784314e+00]
>face is a Match (0.490 <= 0.500)
[0.8692604  0.21581109 0.786859

In [44]:
[x[0] for x in results if x[1]]

['caizhuoyan',
 'chenbailin',
 'chenderong',
 'chenguanxi',
 'chenhe',
 'chenhuilin',
 'chenxiang',
 'chenyixun',
 'dongjie',
 'dongxuan',
 'gaoyixiang',
 'gongbeibi',
 'gutianle',
 'hanqingzi',
 'huangjinglun',
 'huhaiquan',
 'hujing',
 'huojianhua',
 'huyanbin',
 'jiangjinfu',
 'jingtian',
 'jujingyi',
 'licaihua',
 'lilianjie',
 'linjing',
 'linyichen',
 'linyoujia',
 'liruotong',
 'liudehua',
 'liutao',
 'liuye',
 'liyapeng',
 'lufangsheng',
 'lvjiarong',
 'make',
 'meiyanfang',
 'mowenwei',
 'nieyuan',
 'qianyongchen',
 'qinhao',
 'qiuze',
 'raowei',
 'ruanjingtian',
 'songqian',
 'sunyaowei',
 'tangning',
 'tongkeke',
 'wangbaoqiang',
 'wangkai',
 'wangou',
 'wenghong',
 'wujunyu',
 'xiongnaijin',
 'xuhao',
 'xujiao',
 'yanggongru',
 'yangmi',
 'yanyalun',
 'yeqianwen',
 'yeyiqian',
 'yingcaier',
 'yuanshanshan',
 'yuanyongyi',
 'zhangbaizhi',
 'zhangbo',
 'zhangdansan',
 'zhanghan',
 'zhanghanyun',
 'zhangyi',
 'zhangyishan',
 'zhangzhilin',
 'zhangzifeng',
 'zhaojiamin',
 'zhao

In [34]:
results_aidai = []

unmasked_aidai = sorted(list((unmasked_dataset_path / 'aidai').iterdir()))
masked_aidai = sorted(list((masked_dataset_path / 'aidai').iterdir()))[0]
for unmasked_aidai_image in unmasked_aidai:
    unmasked_image_path = str(unmasked_aidai_image)
    masked_image_path = str(masked_aidai)
    
    unmasked_features, masked_features = get_embeddings([unmasked_image_path, masked_image_path])
    matched = is_match(unmasked_features, masked_features)
    
    results_aidai.append(matched)

[6.4286904  0.891206   0.         ... 0.01346317 3.2546344  0.03368115] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.634 > 0.500)
[4.5168366 0.4488416 0.        ... 1.2699113 3.4065237 1.365218 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.681 > 0.500)
[8.54877    0.33906832 0.16439788 ... 0.6162315  1.3641261  3.298376  ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.680 > 0.500)
[6.872231   0.05894016 0.         ... 0.04032178 0.12371155 0.01964294] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.623 > 0.500)
[7.2152214  0.3033258  0.43158862 ... 0.01459123 1.6966288  3.1674106 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.684 > 0.500)
[7.292639   0.16341522 0.         ... 0.         7.0192084  0.0668417 ] 
 [0.2876734 0.        0.        ... 0.        1.

[13.487495    0.02586578  0.         ...  8.637402    9.988133
  0.        ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.734 > 0.500)
[2.2625926  0.73338974 0.04086011 ... 4.825842   0.41985413 0.11194608] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.702 > 0.500)
[4.044325   0.47735083 0.         ... 0.         1.5229471  4.013964  ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.660 > 0.500)
[5.8371606  0.12582676 0.         ... 0.         3.187961   1.3176261 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.589 > 0.500)
[10.243253    1.9388317   0.         ...  0.30063584  2.128215
  0.        ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.626 > 0.500)
[6.7274394  1.1319431  0.05108449 ... 2.1457276  3.3923726  1.0687448 ] 
 [0.2876734 0.        0.        

[7.7142415  1.0165849  0.00926682 ... 2.4766366  3.3948636  2.0895612 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.646 > 0.500)
[9.5171585  0.6078052  0.         ... 0.79111636 0.85043913 1.6897597 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.577 > 0.500)
[0.6240741  0.17297374 7.171658   ... 5.422498   0.40380198 1.6163886 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.672 > 0.500)
[4.0264225  0.26042044 0.0407026  ... 0.         1.254674   4.5101323 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.660 > 0.500)
[0.         0.01556253 8.496543   ... 0.47122672 1.2474961  0.0976609 ] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.645 > 0.500)
[1.8624496  0.         0.02054922 ... 0.         0.34403837 5.217421  ] 
 [0.2876734 0.        0.        ... 0.    

[3.255824   1.5887455  0.05077593 ... 0.28502423 2.4377031  0.44354716] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.672 > 0.500)
[12.173669    1.2751988   0.         ...  1.7715254   0.07109825
  0.30367276] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.625 > 0.500)
[1.1791639 0.526101  0.        ... 0.0592992 3.7021427 0.7636353] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.551 > 0.500)
[9.4970131e+00 5.9697199e-01 0.0000000e+00 ... 2.7477206e-03 8.6511507e+00
 1.5375217e+00] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.603 > 0.500)
[4.4577236  0.7894732  0.         ... 1.6509304  2.2525916  0.06124623] 
 [0.2876734 0.        0.        ... 0.        1.1830741 0.5324675]
>face is NOT a Match (0.634 > 0.500)
[7.5837097  1.5474625  0.         ... 0.06393143 0.9078764  0.        ] 
 [0.2876734 0.        

In [36]:
any(results_aidai)

False

In [57]:
model.fit

<bound method Model.fit of <keras.engine.functional.Functional object at 0x000001B802203B20>>

In [58]:
people = {}

unmasked_people = unmasked_dataset_path.iterdir()
for unmasked_person in unmasked_people:
    masked_person = masked_dataset_path / unmasked_person.name
    if not masked_person.is_dir():
        continue
    unmasked_image_count = len(list(unmasked_person.iterdir()))
    masked_image_count = len(list(masked_person.iterdir()))
    people[unmasked_person.name] = (unmasked_image_count, masked_image_count)

In [61]:
len([name for name,(unmasked,masked) in people.items() if unmasked >=8 and masked >= 8])

52

In [ ]:
import cv2
cv2.norm(x, y, cv2.NORM_L2)